In [2]:
from datetime import datetime, date, time, timedelta
import pandas as pd, numpy as np
from matplotlib import pyplot as plt

In [3]:
n = 1
dist = 1360 * n
kml_G = 13.6
price_LG = 7.9
kml_E = 12.0
price_LE = 5.7
capacity = 50
authonomy_G = capacity * kml_G
authonomy_E = capacity * kml_E
dist_authonomy_G = dist / authonomy_G
dist_authonomy_E = dist / authonomy_E

In [4]:
# Calc
km100_E = 100 / kml_E
km100_G = 100 / kml_G
qt_liters_G = dist / 100 * km100_G
qt_liters_E = dist / 100 * km100_E
value_Km_G = price_LG / kml_G
value_Km_E = price_LE / kml_E
value_100Km_G = 100 / kml_G * price_LG
value_100Km_E = 100 / kml_E * price_LE
value_G = price_LG * dist / 100 * km100_G
value_E = price_LE * dist / 100 * km100_E

In [5]:
df_fuel = pd.DataFrame(
	{
		'Fuel'				: ['Gasoline', 'Ethanol'],
		'Distance (Km)'		: [dist, dist],
		'Capacity (L)'		: [capacity, capacity],
		'R$/L'				: [price_LG, price_LE],
		'R$/fill'			: [price_LG * capacity, price_LE * capacity],
		'Authonomy (Km)'	: [authonomy_G, authonomy_E],
		'Km/authonomy'		: [dist_authonomy_G, dist_authonomy_E],
		'Km/L'				: [kml_G, kml_E],
		'L/100 Km'			: [km100_G, km100_E],
		'Value/Km'			: [value_Km_G, value_Km_E],
		'R$/100 Km'			: [value_100Km_G, value_100Km_E],
		'Total fuel (L)'	: [qt_liters_G, qt_liters_E],
		'Total value (R$)'	: [value_G, value_E],
		'R$ ratio'			: [price_LG / price_LE, price_LE / price_LG],
		'Km ratio'			: [kml_G / kml_E, kml_E / kml_G]
	}
)
df_fuel

,Fuel,Distance (Km),Capacity (L),R$/L,R$/fill,Authonomy (Km),Km/authonomy,Km/L,L/100 Km,Value/Km,R$/100 Km,Total fuel (L),Total value (R$),R$ ratio,Km ratio
0,Gasoline,1360,50,7.9,395.0,680.0,2.000000,13.6,7.352941,0.580882,58.088235,100.000000,790.0,1.385965,1.133333
1,Ethanol,1360,50,5.7,285.0,600.0,2.266667,12.0,8.333333,0.475000,47.500000,113.333333,646.0,0.721519,0.882353


In [6]:
one_way = int(dist / n)
avg_speed = 80

In [7]:
df_trip = pd.DataFrame(
	{
		'Distance (Km)'		: [one_way],
		'AVG speed (Km/h)'	: [avg_speed],
		'Departure time'	: [datetime.combine(date.today(), time(7, 0, 0))]
	}
)
df_trip['Arrival time'] = df_trip['Departure time'] + timedelta(seconds = int((df_trip['Distance (Km)'] / df_trip['AVG speed (Km/h)']) * 3600))
df_trip['Elapsed time'] = df_trip['Arrival time'] - df_trip['Departure time']
d = pd.to_timedelta(df_trip['Elapsed time']).dt.components[['days', 'hours', 'minutes', 'seconds']]
d['hours'] = d['hours'].add(d.pop('days') * 24)

df_trip['Elapsed time hh:mm:ss'] = d.astype(str).agg(lambda s: ':'.join(s.str.zfill(2)), axis=1)

df_trip

,Distance (Km),AVG speed (Km/h),Departure time,Arrival time,Elapsed time,Elapsed time hh:mm:ss
0,1360,80,2022-04-27 07:00:00,2022-04-28,0 days 17:00:00,17:00:00
